In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [2]:
# Network with 2 conv. layers and 3 linear layers
class Network2Con3Lin(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        # Linear layers
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
        
    def forward(self, t):
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, 2, 2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, 2, 2)
        
        t = t.reshape(-1, 12*4*4) # Flatten image after conv. layers
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t
     
# Network with 3 conv. layers and 4 linear layers
class Network3Con4Lin(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=9, kernel_size=2)
        self.conv3 = nn.Conv2d(in_channels=9, out_channels=12, kernel_size=2)
        
        # Linear layers
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=180)
        self.fc2 = nn.Linear(in_features=180, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
        
    def forward(self, t):
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, 2, 2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, 2, 2)
        
        t = F.relu(self.conv3(t))
        t = F.max_pool2d(t, kernel_size=2, stride=1)
        
        t = t.reshape(-1, 12*4*4) # Flatten image after conv. layers
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = F.relu(self.fc3(t))
        t = self.out(t)
        
        return t
    
# Network with 1 conv. layer and 2 linear layers
class Network1Con2Lin(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=9)
        
        # Linear layers
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.out = nn.Linear(in_features=120, out_features=10)
        
        
    def forward(self, t):
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, 5, 5)
        
        t = t.reshape(-1, 12*4*4) # Flatten image after conv. layer
        t = F.relu(self.fc1(t))
        t = self.out(t)
        
        return t

In [3]:
# Returns the number of correct predictions for a given set of predictions and labels
def get_num_correct(predictions, labels):
    return predictions.argmax(dim=1).eq(labels).sum().item()

In [4]:
# Evaluates a given network and names the datafile accordingly
def evaluate_network(network, run_name):
    
    optimizer = optim.Adam(network.parameters(), lr=0.01)
    
    # Create the dataloader for the data set with batch size 100
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)
    
    # Create a SummaryWriter for recording data files
    tb = SummaryWriter(run_name)
    
    # Record the total number of batches and correct predictions from the network
    total_correct = 0
    batchNo = 0
    
    # Create an empty confusion matrix
    confusion_mat = torch.zeros(10, 10, dtype=torch.int64)
    
    
    # Pass all tensors to the network in batches of 100 images
    for batch in dataloader:
    
        images, labels = batch
        
        # Get predictions and loss from network
        predictions = network(images)
        loss = F.cross_entropy(predictions, labels)
        
        # Update network
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update total number of correct predictions
        total_correct += get_num_correct(predictions, labels)
    
        # Add datapoint to SummaryWriter object
        tb.add_scalar('Correct Percentage', total_correct/((batchNo+1)*100), batchNo)
        
        # Update confusion matrix
        preds_max = predictions.argmax(dim=1).numpy()
        labels_arr = labels.numpy()
        
        for pred_idx in range(100):
            pred_val = preds_max[pred_idx]
            pred_true = labels_arr[pred_idx]
            confusion_mat[pred_true, pred_val] = confusion_mat[pred_true, pred_val] + 1
            
        batchNo += 1
    
        
    print("total correct:", total_correct) # Print total correct predictions
    tb.close() # Close SummaryWriter object to save the data file
    print(confusion_mat) # Display confusion matrix
    
    
    
    return

In [5]:
# Creates the dataset from the Fashion MNIST dataset included in torchvision
dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train = True,
    download=True,
    transform=transforms.Compose([
            transforms.ToTensor()
    ])
)

dataset_labels = dataset.targets

In [6]:
network = Network2Con3Lin()
evaluate_network(network, 'runs/Two Convolutional Three Linear')


network2 = Network3Con4Lin()
evaluate_network(network2, 'runs/Three Convolutional Four Linear')


network3 = Network1Con2Lin()
evaluate_network(network3, 'runs/One Convolutional Two Linear')


total correct: 47330
tensor([[4664,   79,  105,  437,   78,   39,  497,    2,   81,   18],
        [  33, 5670,   12,  215,   29,   13,   19,    0,    4,    5],
        [  94,   24, 3829,   55, 1109,   27,  731,    2,   78,   51],
        [ 332,  171,   49, 4906,  296,   23,  184,    1,   26,   12],
        [  46,   42,  713,  299, 4360,   26,  420,    0,   50,   44],
        [   1,    4,    0,    9,    3, 5386,    0,  340,   54,  203],
        [1228,   44, 1231,  256,  888,   42, 2155,    0,  126,   30],
        [   0,    7,    0,    0,    2,  307,    0, 5258,   13,  413],
        [  31,   25,   90,   42,   71,   89,   78,   17, 5500,   57],
        [   1,    7,    1,    4,   13,   85,    1,  277,    9, 5602]])
total correct: 39255
tensor([[4074,  200,  151,  595,  135,    9,  534,   38,  147,  117],
        [ 355, 4657,   37,  544,   81,    1,  128,   33,   64,  100],
        [ 233,   42, 2888,  140, 1556,    7,  846,   47,  140,  101],
        [ 595,  576,   72, 3756,  379,   10,  3